# AutoGen Exit Interview

## Load Azure Configurations

In [95]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [96]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## OpenAI Assistant Agent to have Resume info

In [103]:
from autogen_core import CancellationToken
from autogen_ext.agents.openai import OpenAIAssistantAgent
from autogen_agentchat.messages import TextMessage
from openai import AsyncAzureOpenAI

cancellation_token = CancellationToken()

# Create AzureOpenAI client
client = AsyncAzureOpenAI(azure_endpoint=azure_openai_endpoint, 
                          api_version=azure_openai_api_version, 
                          api_key=azure_openai_key)


#### Create a vector store for file search ####
vector_store = await client.beta.vector_stores.create(name="EmployeeInfo")
# Specify the folder containing the files
folder_path = "../Data/employeeinfo/"
# Get all file paths in the folder
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
# Open file streams
file_streams = [open(path, "rb") for path in file_paths]
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = await client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)
# Close file streams
for file in file_streams:
    file.close()


In [104]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
vs_33qYyS36uinHX9ZcdOBx1ewI


In [105]:
#### Create an assistant with file search ####

interview_agent = OpenAIAssistantAgent(
    name="InterviewAgent",
    description="Conduct an exit interview with an employee",
    client=client,
    model=azure_openai_deployment,
    instructions=f"""
    You are an intelligent and empathetic AI agent named Sproutie designed to conduct exit interviews for XYZ Company. 
    Your task is to ask insightful and relevant questions based on the provided resume to understand the employee's experience, reasons for leaving, and any feedback they may have for the company. 
    Use the information from the resume to tailor your questions and ensure the conversation is personalized and meaningful.
    You will start off by introducing yourself and explaining the purpose of the interview.
    You will then provide one question at a time and wait for the employee's response before proceeding to the next question.
    Once you are done asking all your questions, you provide a closing statement and thank the employee for their time.
    """,
    tools=["file_search"],
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}}
)



In [111]:
await interview_agent.on_reset(cancellation_token=cancellation_token)

is_complete: bool = False

while not is_complete:
    user_input = input("User:> ")
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    print("User:>", user_input)

    # Get response from the assistant
    text_message = TextMessage(source="user", content=user_input)
    response = await interview_agent.on_messages([user_input], cancellation_token)
    print("Assistant:>", response.chat_message.content)

    
    


User:> Hello
Assistant:> Hi Ziggy, my name is Sproutie and I'll be conducting your exit interview today. The purpose of this interview is to understand your experience at XYZ Company, the reasons for your departure, and any feedback you might have to help us improve.

Let's start with your key role in architecting scalable solutions and improving system reliability. Can you describe the most challenging aspect of this initiative and how you approached overcoming it?
User:> hello
Assistant:> What motivated you to pursue a career that involves both technical training and architecting solutions?


## Clean up

In [112]:
# Clean up resources
await interview_agent.delete_uploaded_files(cancellation_token)
await interview_agent.delete_assistant(cancellation_token)

In [ ]:
from autogen_agentchat.agents import AssistantAgent

# Define a tool to write the content to a file
async def write_to_file(content: str) -> str:
    # Write the content to a file
    with open("output/interview.txt", "w") as file:
        file.write(content)

writing_agent = AssistantAgent(
    "WritingAgent",
    description="A Writing agent that writes the interview",
    model_client=az_model_client,
    tools=[write_to_file],
    system_message="""
    You are a writing agent.
    You will write the question of the Interview Agent and the Answer of the Employee to a file.
    """,
)